# Wasm Chat

`Wasm-chat` allows you to chat with LLMs of [GGUF](https://github.com/ggerganov/llama.cpp/blob/master/gguf-py/README.md) format both locally and via chat service.


`Wasm-chat` is driven by [WasmEdge Runtime](https://wasmedge.org/), a popular WebAssmbly runtime. It is also an attempt to run AI tasks within wasm containers.

## Chat Locally

### Setup

To deploy WasmEdge Runtime and `wasm-infer.wasm` in your local system, run the following command in your terminal:

In [1]:
!curl -sSf https://raw.githubusercontent.com/second-state/wasm-llm/main/deploy.sh | bash


The installation will deploy 'WasmEdge Runtime' and 'wasm-infer.wasm' in your local environment:

[+] Checking the operating system ...

[+] Checking if 'git' and 'curl' are installed ...

[+] Installing WasmEdge ...

Using Python: /home/ubuntu/miniconda3/envs/langchain/bin/python3 
ERROR   - Exception on process - rc= 127 output= b'' command= ['/usr/local/cuda/bin/nvcc --version 2>/dev/null']
WARNING - Experimental Option Selected: plugins
WARNING - plugins option may change later
INFO    - Compatible with current configuration
INFO    - Running Uninstaller
WARNING - SHELL variable not found. Using bash as SHELL
INFO    - shell configuration updated
INFO    - Downloading WasmEdge
|============================================================|100.00 %INFO    - Downloaded
INFO    - Installing WasmEdge
INFO    - WasmEdge Successfully installed
INFO    - Downloading Plugin: wasi_nn-ggml
|============================================================|100.00 %INFO    - Downloaded
INFO    - Ru

### Get model

For the purpose of demonstration, we use a smaller model `TinyLlama-1.1B-Chat-v0.3`. Run the following command to download the model:

In [2]:
!curl -LO https://huggingface.co/second-state/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q5_K_M.gguf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1176  100  1176    0     0   5909      0 --:--:-- --:--:-- --:--:--  5939
100  745M  100  745M    0     0   232M      0  0:00:03  0:00:03 --:--:--  272M


More models and sample code can be found in [GGUF Models](https://github.com/second-state/wasm-llm/blob/main/MODEL.md).

### Code: One-turn Conversation

The following code shows a one-turn conversation with the model:

In [2]:
from langchain.chat_models.wasm_chat import ChatWasmLocal, PromptTemplateType
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage

model_file = "/Volumes/Store/models/gguf/tinyllama-1.1b-chat-v0.3.Q5_K_M.gguf"

chat = ChatWasmLocal(model_file=model_file, prompt_template=PromptTemplateType.ChatML)

system_message = SystemMessage(content="You are an AI assistant")
user_message = HumanMessage(content="What is the capital of France?")
messages = [system_message, user_message]
chat_result = chat(messages)
print(f"[Bot] {chat_result.content}")

assert isinstance(chat_result, AIMessage)
assert isinstance(chat_result.content, str)
assert "Paris" in chat_result.content

[Bot] Paris


## Chat via API Service

Compare with chat locally, chat via API service is more convenient. You don't need to install WasmEdge Runtime and download the model. You can chat with the model by sending a HTTP request to the API service.

More importantly, by following the steps in [README](https://github.com/second-state/llama-utils/tree/main/api-server#readme), you can host your own API service so that you can chat with any models you like on any device you have anywhere as long as the internet is available.

In [3]:
from langchain.chat_models.wasm_chat import ChatWasmService
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage

ip_addr = "0.0.0.0" # change it to your own service ip address
port = "8080" # change it to your own service port

chat = ChatWasmService(service_ip_addr=ip_addr, service_port=port)
system_message = SystemMessage(content="You are an AI assistant")
user_message = HumanMessage(content="What is the capital of France?")
messages = [system_message, user_message]
response = chat(messages)
print(f"[Bot] {chat_result.content}")

assert isinstance(chat_result, AIMessage)
assert isinstance(chat_result.content, str)
assert "Paris" in chat_result.content

ValueError: Error code: 502, reason: Bad Gateway